In [1]:
# !pip install geopandas shapely pyarrow pandas requests tqdm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 18.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ------------- -------------------------- 6.3/19.2 MB 48.2 MB/s eta 0:00:01
   ---------------- ----------------------- 7.9/19.2 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------- 19.2/19.2 MB 33.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 6.1/6.1 MB 92.5 MB/s eta 0:00:00


In [8]:
import os
import zipfile
from io import BytesIO
from pathlib import Path

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests


INPUT_CSV = "I:/Data_for_practice/Rfiles/FundUS/nsf_aff.csv"
OUTPUT_CSV = "I:/Data_for_practice/Rfiles/FundUS/nsf_aff_with_tract_names.csv"
CACHE_DIR = "./tiger_cache"
TIGER_YEAR = 2023


STATEFP_TO_NAME = {
    "01": "Alabama", "02": "Alaska", "04": "Arizona", "05": "Arkansas", "06": "California",
    "08": "Colorado", "09": "Connecticut", "10": "Delaware", "11": "District of Columbia",
    "12": "Florida", "13": "Georgia", "15": "Hawaii", "16": "Idaho", "17": "Illinois",
    "18": "Indiana", "19": "Iowa", "20": "Kansas", "21": "Kentucky", "22": "Louisiana",
    "23": "Maine", "24": "Maryland", "25": "Massachusetts", "26": "Michigan", "27": "Minnesota",
    "28": "Mississippi", "29": "Missouri", "30": "Montana", "31": "Nebraska", "32": "Nevada",
    "33": "New Hampshire", "34": "New Jersey", "35": "New Mexico", "36": "New York",
    "37": "North Carolina", "38": "North Dakota", "39": "Ohio", "40": "Oklahoma",
    "41": "Oregon", "42": "Pennsylvania", "44": "Rhode Island", "45": "South Carolina",
    "46": "South Dakota", "47": "Tennessee", "48": "Texas", "49": "Utah", "50": "Vermont",
    "51": "Virginia", "53": "Washington", "54": "West Virginia", "55": "Wisconsin", "56": "Wyoming",
    "60": "American Samoa", "66": "Guam", "69": "Northern Mariana Islands", "72": "Puerto Rico", "78": "U.S. Virgin Islands"
}


def download_extract(url: str, out_dir: str) -> str:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # 이미 shp 있으면 스킵
    if any(out_dir.glob("*.shp")):
        return str(out_dir)

    r = requests.get(url, timeout=240)
    r.raise_for_status()
    with zipfile.ZipFile(BytesIO(r.content)) as z:
        z.extractall(out_dir)

    return str(out_dir)


def tiger_tract_url(year: int, statefp: str) -> str:
    # Tract는 "주 단위" 파일만 존재
    return f"https://www2.census.gov/geo/tiger/TIGER{year}/TRACT/tl_{year}_{statefp}_tract.zip"


def load_state_tracts(year: int, statefp: str, cache_dir: str) -> gpd.GeoDataFrame:
    folder = Path(cache_dir) / f"tl_{year}_{statefp}_tract"
    download_extract(tiger_tract_url(year, statefp), folder)

    shp = next(folder.glob("*.shp"), None)
    if shp is None:
        raise FileNotFoundError(f"Tract shapefile not found in {folder}")

    gdf = gpd.read_file(shp).to_crs(epsg=4326)

    # NAMELSAD = 사람이 읽는 tract 이름(예: "Census Tract 1.01")
    keep = ["GEOID", "STATEFP", "COUNTYFP", "TRACTCE", "NAMELSAD", "geometry"]
    gdf = gdf[keep].rename(
        columns={
            "GEOID": "tract_geoid",
            "STATEFP": "statefp",
            "COUNTYFP": "countyfp",
            "TRACTCE": "tractce",
            "NAMELSAD": "tract_name",
        }
    )
    return gdf


def load_us_counties(year: int, cache_dir: str) -> gpd.GeoDataFrame:
    # County는 전국 단일 파일이 존재
    url = f"https://www2.census.gov/geo/tiger/TIGER{year}/COUNTY/tl_{year}_us_county.zip"
    folder = Path(cache_dir) / f"tl_{year}_us_county"
    download_extract(url, folder)

    shp = next(folder.glob("*.shp"), None)
    if shp is None:
        raise FileNotFoundError(f"County shapefile not found in {folder}")

    gdf = gpd.read_file(shp).to_crs(epsg=4326)
    gdf = gdf[["STATEFP", "COUNTYFP", "NAME", "geometry"]].rename(
        columns={"STATEFP": "statefp", "COUNTYFP": "countyfp", "NAME": "county_name"}
    )
    # 조인 키(문자형 보장)
    gdf["statefp"] = gdf["statefp"].astype(str).str.zfill(2)
    gdf["countyfp"] = gdf["countyfp"].astype(str).str.zfill(3)
    gdf["county_geoid"] = gdf["statefp"] + gdf["countyfp"]
    return gdf[["county_geoid", "county_name"]]


def add_tract_and_names(df: pd.DataFrame, lat_col="Latitude", lon_col="Longitude",
                        year: int = 2023, cache_dir: str = "./tiger_cache") -> pd.DataFrame:

    out = df.copy()
    out[lat_col] = pd.to_numeric(out[lat_col], errors="coerce")
    out[lon_col] = pd.to_numeric(out[lon_col], errors="coerce")

    # 결과 컬럼 준비
    for c in ["tract_geoid", "tractce", "tract_name", "statefp", "state_name", "countyfp", "county_name"]:
        out[c] = pd.NA

    # 포인트 생성
    pts = gpd.GeoDataFrame(
        out,
        geometry=[
            Point(xy) if pd.notna(xy[0]) and pd.notna(xy[1]) else None
            for xy in zip(out[lon_col], out[lat_col])
        ],
        crs="EPSG:4326",
    )
    valid = pts[pts.geometry.notna()].copy()
    if len(valid) == 0:
        return out

    # 1) 먼저 County 폴리곤으로 statefp 후보를 좁혀서 tract 다운로드 최소화
    #    (county는 전국 파일이 있으니 이를 사용)
    counties_poly = gpd.read_file(
        f"https://www2.census.gov/geo/tiger/TIGER{year}/COUNTY/tl_{year}_us_county.zip"
    ).to_crs(epsg=4326)[["STATEFP", "geometry"]]

    tmp = gpd.sjoin(valid, counties_poly, how="left", predicate="within")
    statefps = pd.Series(tmp["STATEFP"].dropna().astype(str).str.zfill(2)).unique().tolist()

    # 2) 해당 state들의 tract만 로딩해서 합치기
    tract_frames = []
    for s in statefps:
        tract_frames.append(load_state_tracts(year, s, cache_dir))
    tracts = pd.concat(tract_frames, ignore_index=True)

    # 3) Tract 매칭
    joined = gpd.sjoin(
        valid, tracts, how="left", predicate="within",
        lsuffix="pt", rsuffix="tract"
    )

    out.loc[joined.index, "tract_geoid"] = joined["tract_geoid_tract"].values
    out.loc[joined.index, "tractce"] = joined["tractce_tract"].values
    out.loc[joined.index, "tract_name"] = joined["tract_name_tract"].values
    out.loc[joined.index, "statefp"] = pd.Series(joined["statefp_tract"]).astype(str).str.zfill(2).values
    out.loc[joined.index, "countyfp"] = pd.Series(joined["countyfp_tract"]).astype(str).str.zfill(3).values

    # 4) State 이름 붙이기
    out["state_name"] = out["statefp"].map(STATEFP_TO_NAME)

    # 5) County 이름 붙이기 (STATEFP+COUNTYFP로 merge)
    counties_names = load_us_counties(year, cache_dir)
    out["county_geoid"] = out["statefp"].astype("string") + out["countyfp"].astype("string")
    out = out.merge(counties_names, on="county_geoid", how="left", suffixes=("", "_from_county"))

    # merge 결과 정리: county_name 최종 확정
    out["county_name"] = out["county_name"].fillna(out["county_name_from_county"])
    out = out.drop(columns=[c for c in ["county_name_from_county", "county_geoid"] if c in out.columns])

    return out


def main():
    df = pd.read_csv(INPUT_CSV)
    df2 = add_tract_and_names(df, lat_col="Latitude", lon_col="Longitude",
                              year=TIGER_YEAR, cache_dir=CACHE_DIR)
    df2.to_csv(OUTPUT_CSV, index=False)
    print("Saved:", OUTPUT_CSV)
    print("Matched tracts:", df2["tract_geoid"].notna().sum())


if __name__ == "__main__":
    main()


Saved: I:/Data_for_practice/Rfiles/FundUS/nsf_aff_with_tract_names.csv
Matched tracts: 9247
